In [22]:
import os
from dotenv import load_dotenv
from google.analytics.data_v1beta import BetaAnalyticsDataClient 
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
from google.oauth2 import service_account
import sqlite3

# LOADING PROPERTY_ID AND GA4 CREDENTIALS JSON FILE
#load_dotenv(r"C:\Users\garvi\Documents\Data Science Projects\Digital Marketing Dashboard\.env")
PROPERTY_ID = os.getenv("Property_Id")
CREDENTIALS_GA4_JSON = os.getenv("GA4_Service_Account_Credentials", r"C:\Users\garvi\Documents\Data Science Projects\Digital Marketing Dashboard\eastern-perigee-469018-v9-ac5502363b52.json" )

print(os.getenv("GA4_Service_Account_Credentials"))
print(os.getenv("Property_Id"))

# intializing client , request and responmse to fetch data from google analytics 4 with property Id and service credentials json

credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_GA4_JSON)
client = BetaAnalyticsDataClient(credentials = credentials)

request = RunReportRequest(
    property  = f"properties/{PROPERTY_ID}",
    dimensions = [  Dimension(name="date"),
                    Dimension(name="city"),
                    Dimension(name="sessionDefaultChannelGroup")],
    metrics = [Metric(name="activeUsers"),
               Metric(name="averageSessionDuration"),
               Metric(name="newUsers"),
              Metric(name="engagementRate"),
               Metric(name="userEngagementDuration"),
              Metric(name="eventCount")],
    date_ranges = [DateRange(start_date = "2025-08-14", end_date = "2025-08-20")]
)

response = client.run_report(request)
#print(response)

for row in response.rows:
    print([v.value for v in row.dimension_values],[v.value for v in row.metric_values])
    
 

conn = sqlite3.connect("mydatabase.db", timeout=10)
cursor = conn.cursor()
    
# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS ga4_detailed_stats (
    date TEXT,
    city TEXT,
    channel_group TEXT,
    sessions INTEGER,
    new_users INTEGER,
    engagement_rate REAL,
    avg_session_duration REAL,
    engagement_duration REAL,
    event_count INTEGER
)
""")

# Insert rows
for row in response.rows:
    date = row.dimension_values[0].value
    city = row.dimension_values[1].value
    channel_group = row.dimension_values[2].value

    sessions = int(row.metric_values[0].value or 0)
    new_users = float(row.metric_values[1].value or 0)
    engagement_rate = float(row.metric_values[2].value or 0)
    avg_session_duration = float(row.metric_values[3].value or 0)
    engagement_duration = float(row.metric_values[4].value or 0) / 1000  # convert ms → sec
    event_count = int(row.metric_values[5].value or 0)

    cursor.execute("""
    INSERT OR IGNORE INTO ga4_detailed_stats VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (date, city, channel_group, sessions, new_users,
          engagement_rate, avg_session_duration, engagement_duration, event_count))



# 1. App Analytics (daily data)
cursor.execute("""
CREATE TABLE IF NOT EXISTS app_analytics (
    date TEXT PRIMARY KEY,
    total_sales REAL,
    signups INTEGER,
    active_users INTEGER,
    sessions INTEGER,
    engagement_rate REAL
)
""")

# Insert demo data
app_data = [
    ("2025-08-15", 1200.50, 45, 300, 500, 0.65),
    ("2025-08-16", 1500.75, 52, 340, 560, 0.72),
    ("2025-08-17", 980.00, 38, 280, 450, 0.60),
    ("2025-08-18", 1700.25, 60, 390, 620, 0.75),
    ("2025-08-19", 1350.40, 48, 320, 510, 0.68),
]

cursor.executemany("""
INSERT OR REPLACE INTO app_analytics VALUES (?, ?, ?, ?, ?, ?)
""", app_data)


# 2. Google Ads (campaign-level data)
cursor.execute("""
CREATE TABLE IF NOT EXISTS google_ads (
    campaign_id INTEGER PRIMARY KEY,
    campaign_name TEXT,
    date TEXT,
    impressions INTEGER,
    clicks INTEGER,
    cost REAL,
    conversions INTEGER
)
""")

# Insert demo data
ads_data = [
    (1, "Brand Campaign", "2025-08-15", 5000, 320, 150.75, 40),
    (2, "Search Campaign", "2025-08-15", 8000, 500, 220.10, 55),
    (3, "Display Campaign", "2025-08-15", 12000, 280, 180.00, 25),
    (1, "Brand Campaign", "2025-08-16", 5200, 340, 160.00, 42),
    (2, "Search Campaign", "2025-08-16", 7500, 480, 210.50, 50),
    (3, "Display Campaign", "2025-08-16", 13000, 300, 190.25, 28),
]

cursor.executemany("""
INSERT OR REPLACE INTO google_ads VALUES (?, ?, ?, ?, ?, ?, ?)
""", ads_data)

cursor.execute("select * from google_ads")
rows = cursor.fetchall()
for row in rows:
    print(row)


cursor.execute("select * from app_analytics")
rows = cursor.fetchall()
for row in rows:
    print(row)

   
conn.commit()
conn.close()
    



None
501340180
['20250815', 'Hansi', 'Direct'] ['5', '769.90745171428568', '5', '0.5714285714285714', '75', '118']
['20250815', 'Delhi', 'Direct'] ['1', '699.075729', '0', '1', '11', '9']
['20250815', 'Gharaunda', 'Direct'] ['1', '18.82465', '1', '1', '18', '5']
(1, 'Brand Campaign', '2025-08-16', 5200, 340, 160.0, 42)
(2, 'Search Campaign', '2025-08-16', 7500, 480, 210.5, 50)
(3, 'Display Campaign', '2025-08-16', 13000, 300, 190.25, 28)
('2025-08-15', 1200.5, 45, 300, 500, 0.65)
('2025-08-16', 1500.75, 52, 340, 560, 0.72)
('2025-08-17', 980.0, 38, 280, 450, 0.6)
('2025-08-18', 1700.25, 60, 390, 620, 0.75)
('2025-08-19', 1350.4, 48, 320, 510, 0.68)
